In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import os, json
import numpy as np
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
import joblib
import cmaes

def preprocess_for_nn(v_sim, j_sim, target_v_start, target_v_end, target_points=128):
    """
    v_sim: original voltage array (e.g., -10 to 10)
    j_sim: original current density array
    """
    # 1. Define target grids
    v_fwd_target = np.logspace(np.log10(target_v_start), np.log10(target_v_end), target_points)
    v_rev_target = - v_fwd_target # Negative sequence
    
    # 2. Create Interpolation function
    # Using 'linear' or 'cubic'; linear is safer for simulation data
    interp_func = interp1d(v_sim, j_sim, kind='linear', fill_value="extrapolate")
    
    # 3. Interpolate
    j_fwd = interp_func(v_fwd_target)
    j_rev = interp_func(v_rev_target)
    
    j_combined = np.concatenate([j_fwd, j_rev])
    
    # 4. Transform: Absolute -> Log
    # Clip to avoid log(0) - adjust 1e-10 to your noise floor
    j_abs = np.abs(j_combined)
    j_log = np.log(np.clip(j_abs, 1e-10, None))
    
    return j_log

cwd = os.getcwd()
mydrive_path = '/content/drive/MyDrive/'
model_path = os.path.join(cwd, 'NN_results', '20260226-121824_d250nm_two_defects','y1', 'model.keras')
regressor = load_model(model_path)
scaler_path = os.path.join(cwd, 'NN_results', '20260226-121824_d250nm_two_defects','20260226-121824_d250nm_two_defects_scaler.joblib')
param_scaler = joblib.load(scaler_path)

dataset_name = '20260225_d250nm_two_defects'
json_path = os.path.join(cwd, 'Datagen_results', dataset_name, 'simulation_metadata.json')
with open(json_path, 'r') as f:
    metadata = json.load(f)
j_min, j_max = metadata['j_min'], metadata['j_max']
n_var = len(metadata['varied_parameters'].keys())
lb_mod = np.array([v[0] for v in metadata['varied_parameters_log'].values()])
ub_mod = np.array([v[1] for v in metadata['varied_parameters_log'].values()])

exp_path = os.path.join(cwd, 'expData', '2025-06-05-SOP18n7_JVd_#4_1.dat')
exp_data = np.loadtxt(exp_path, comments = '#')
target_v_start, target_v_end = metadata['target_v_start'], metadata['target_v_end']
combined_v_target = np.tile(np.logspace(np.log10(target_v_start), np.log10(target_v_end), 128), 2)
exp_j_interp_log = preprocess_for_nn(exp_data[:,0], exp_data[:,1], target_v_start, target_v_end) 
voltage_mask = combined_v_target < 100

cost_func_kwargs = {
            "sclc_models": {'sclc_reg': regressor},
            "sclc_scalers": {
                'sclc_stscaler': param_scaler, 'y1_max_sclc': j_max,
                'y1_min_sclc': j_min
            },
            "sclc_data": {
                'y_exp_sclc_log': exp_j_interp_log, 'voltage_mask': voltage_mask
            },
            "sclc_config": {
                'sclc_nn_points': 256, 'sclc_one_jv_len': 128
            }}

In [ ]:
num_run = 1
verbose= True
callbackdata=dict()
obs_mask = [True, True]

print(f"Starting {num_run} optimization run(s)...")

best_overall_error = float('inf')
best_results = {}
for zz in range(num_run):
    if verbose:
        print(f"--- Run {zz + 1}/{num_run} ---")
    _, best_full, best_ln_jvi, best_norm_jvi, error, callbackdata[f"{zz}"] = cmaes.run_cmaes_sclc(
        n_var=n_var,      # self.n_var_jvi,
        lb_mod=lb_mod,    # self.lb_mod_jvi,
        ub_mod=ub_mod,    # self.ub_mod_jvi,
        obs_mask=obs_mask,
        cost_func_kwargs=cost_func_kwargs,
        verbose=verbose)
    # error_ga_all[zz] = error_fit_temp.item()
    if error < best_overall_error:
        best_overall_error = error
        best_results = {
            "best_full": best_full.reshape(1,-1),
            "best_ln": best_ln_jvi,
            "best_norm": best_norm_jvi,
            "error": error
        }
        if verbose:
            print(f"New best error found: {error[0]:.4f}")

# store callbackdata    
for zz in range(num_run):    
    for ii in range(len(callbackdata[f"{zz}"]["generation_offspring"])):
        offspring_ln = (callbackdata[f"{zz}"]["generation_offspring"][ii] * (ub_mod - lb_mod) + lb_mod)
        # (self.ub_mod_jvi-self.lb_mod_jvi) + self.lb_mod_jvi)
        if ii==0 and zz==0:
            ga_suggested_par_mat = offspring_ln
        else:
            ga_suggested_par_mat = np.concatenate((ga_suggested_par_mat,offspring_ln),axis=0) 

print(best_results)   
print("\nOptimization finished.")
print(f"Best error after {num_run} runs: {cmaes['error'][0]:.4f}")